In [ ]:
import pandas as pd

train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DSA/train_LZdllcl.csv")
test  = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DSA/test_2umaH9m.csv")
sample = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DSA/sample_submission_M0L0uXE.csv")




Exploratory Data Analysis

In [ ]:
print("Train shape:", train.shape)
print("Test shape:", test.shape)
print("\nTarget distribution:")
print(train['is_promoted'].value_counts())
print("\nMissing values (train):")
print(train.isnull().sum().sort_values(ascending=False).head(10))
print("\nTrain numeric summary:")
print(train.describe().T[['count','mean','std','min','50%','max']])

Train shape: (54808, 14)
Test shape: (23490, 13)

Target distribution:
is_promoted
0    50140
1     4668
Name: count, dtype: int64

Missing values (train):
previous_year_rating    4124
education               2409
department                 0
region                     0
gender                     0
employee_id                0
recruitment_channel        0
no_of_trainings            0
age                        0
length_of_service          0
dtype: int64

Train numeric summary:
                        count          mean           std   min      50%  \
employee_id           54808.0  39195.830627  22586.581449   1.0  39225.5   
no_of_trainings       54808.0      1.253011      0.609264   1.0      1.0   
age                   54808.0     34.803915      7.660169  20.0     33.0   
previous_year_rating  50684.0      3.329256      1.259993   1.0      3.0   
length_of_service     54808.0      5.865512      4.265094   1.0      5.0   
KPIs_met >80%         54808.0      0.351974      0.477590   0

Preprocessing

In [ ]:
import numpy as np
ID_COL = "employee_id"
TARGET = "is_promoted"

In [ ]:
X_train = train.drop([ID_COL, TARGET], axis=1).copy()
y_train = train[TARGET].copy()
X_test = test.drop([ID_COL], axis=1).copy()

In [ ]:
numeric_cols = X_train.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_cols = [c for c in X_train.columns if c not in numeric_cols]

In [ ]:
num_medians = X_train[numeric_cols].median()
X_train[numeric_cols] = X_train[numeric_cols].fillna(num_medians)
X_test[numeric_cols]  = X_test[numeric_cols].fillna(num_medians)

In [ ]:
for c in categorical_cols:
    X_train[c] = X_train[c].fillna('Unknown').astype(str)
    X_test[c]  = X_test[c].fillna('Unknown').astype(str)

In [ ]:
X_train_ohe = pd.get_dummies(X_train, columns=categorical_cols, drop_first=False)
X_test_ohe  = pd.get_dummies(X_test,  columns=categorical_cols, drop_first=False)

In [ ]:
X_test_ohe = X_test_ohe.reindex(columns=X_train_ohe.columns, fill_value=0)

print("Processed shapes:", X_train_ohe.shape, X_test_ohe.shape)

Processed shapes: (54808, 59) (23490, 59)


Modeling

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import numpy as np

In [ ]:
model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
f1_scores = []

for tr_idx, val_idx in skf.split(X_train_ohe, y_train):
    model.fit(X_train_ohe.iloc[tr_idx], y_train.iloc[tr_idx])
    preds = model.predict(X_train_ohe.iloc[val_idx])
    f1_scores.append(f1_score(y_train.iloc[val_idx], preds))
    print("Logistic Regression CV F1 mean:", np.mean(f1_scores), "std:", np.std(f1_scores))

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression CV F1 mean: 0.3778374420307542 std: 0.0


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression CV F1 mean: 0.3753593989814788 std: 0.0024780430492754024


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression CV F1 mean: 0.37373259378172224 std: 0.0030637866361663017


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression CV F1 mean: 0.37559617028088665 std: 0.004178378096663409
Logistic Regression CV F1 mean: 0.3747314093857431 std: 0.004118048134495149


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Tuning

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rf = RandomForestClassifier(class_weight='balanced', random_state=42, n_jobs=-1)
param_dist = {
    'n_estimators': [100, 200, 400],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}

rs = RandomizedSearchCV(rf, param_distributions=param_dist, n_iter=6,
                        scoring='f1', cv=3, random_state=42, n_jobs=-1, verbose=1)

rs.fit(X_train_ohe, y_train)
print("Best params:", rs.best_params_)
print("Best CV F1 (search):", rs.best_score_)

best_model = rs.best_estimator_

Fitting 3 folds for each of 6 candidates, totalling 18 fits
Best params: {'n_estimators': 400, 'min_samples_split': 5, 'max_depth': None}
Best CV F1 (search): 0.4557978320831891


In [ ]:
best_model.fit(X_train_ohe, y_train)
test_preds = best_model.predict(X_test_ohe).astype(int)
submission = sample.copy()
if 'is_promoted' in submission.columns:
    submission['is_promoted'] = test_preds
else:
    submission.iloc[:, -1] = test_preds
submission.to_csv("/content/drive/MyDrive/Colab Notebooks/DSA/assesment.csv", index=False)
print("Saved submission to  /content/drive/MyDrive/Colab Notebooks/DSA/assesment.csv")
print("Submission head:")
print(submission.head())


Saved submission to  /content/drive/MyDrive/Colab Notebooks/DSA/assesment.csv
Submission head:
   employee_id  is_promoted
0         8724            0
1        74430            0
2        72255            0
3        38562            0
4        64486            0
